In [7]:
import pandas as pd
import numpy as np
from countryinfo import CountryInfo
from geopy.geocoders import Nominatim

import pandas as pd
import numpy as np
import mlforecast
import lightgbm as lgb
from matplotlib import pyplot as plt
import optuna
import plotly_express as px
import seaborn as sns
import os
import gc
import warnings
import statsmodels.api as sm

currency = pd.read_csv('/home/nikl/programming/fhdw/forecasting/ForecastingUniProject/data/Foreign_Exchange_Rates.csv')
train = pd.read_parquet("../data/store_sales/train.parquet")
test = pd.read_parquet("../data/store_sales/test.parquet")
stores = pd.read_parquet("../data/store_sales/stores.parquet")
transactions = pd.read_parquet("../data/store_sales/transactions.parquet").sort_values(["store_nbr", "date"])


train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)

train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")

temp = pd.merge(train.groupby(["date", "store_nbr"]).sales.sum().reset_index(), transactions, how = "left")

print(train.shape)
train = train[~((train.store_nbr == 52) & (train.date < "2017-04-20"))]
train = train[~((train.store_nbr == 22) & (train.date < "2015-10-09"))]
train = train[~((train.store_nbr == 42) & (train.date < "2015-08-21"))]
train = train[~((train.store_nbr == 21) & (train.date < "2015-07-24"))]
train = train[~((train.store_nbr == 29) & (train.date < "2015-03-20"))]
train = train[~((train.store_nbr == 20) & (train.date < "2015-02-13"))]
train = train[~((train.store_nbr == 53) & (train.date < "2014-05-29"))]
train = train[~((train.store_nbr == 36) & (train.date < "2013-05-09"))]
train.shape

print(train.shape)
# Anti Join
c = train.groupby(["store_nbr", "family"]).sales.sum().reset_index().sort_values(["family","store_nbr"])
c = c[c.sales == 0]
outer_join = train.merge(c[c.sales == 0].drop("sales",axis = 1), how = 'outer', indicator = True)
train = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)
del outer_join
gc.collect()
train.shape


/home/nikl/programming/fhdw/forecasting/ForecastingUniProject/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(3000888, 6)
(2780316, 6)


(2698648, 6)

In [8]:
holidays = pd.read_parquet("../data/store_sales/holidays_events.parquet")
holidays["date"] = pd.to_datetime(holidays.date)

# holidays[holidays.type == "Holiday"]
# holidays[(holidays.type == "Holiday") & (holidays.transferred == True)]

# Transferred Holidays
tr1 = holidays[(holidays.type == "Holiday") & (holidays.transferred == True)].drop("transferred", axis = 1).reset_index(drop = True)
tr2 = holidays[(holidays.type == "Transfer")].drop("transferred", axis = 1).reset_index(drop = True)
tr = pd.concat([tr1,tr2], axis = 1)
tr = tr.iloc[:, [5,1,2,3,4]]

holidays = holidays[(holidays.transferred == False) & (holidays.type != "Transfer")].drop("transferred", axis = 1)
holidays = pd.concat([holidays, tr]).reset_index(drop = True)


# Additional Holidays
holidays["description"] = holidays["description"].str.replace("-", "").str.replace("+", "").str.replace('\d+', '')
holidays["type"] = np.where(holidays["type"] == "Additional", "Holiday", holidays["type"])

# Bridge Holidays
holidays["description"] = holidays["description"].str.replace("Puente ", "")
holidays["type"] = np.where(holidays["type"] == "Bridge", "Holiday", holidays["type"])

 
# Work Day Holidays, that is meant to payback the Bridge.
work_day = holidays[holidays.type == "Work Day"]  
holidays = holidays[holidays.type != "Work Day"]  


# Split

# Events are national
events = holidays[holidays.type == "Event"].drop(["type", "locale", "locale_name"], axis = 1).rename({"description":"events"}, axis = 1)

holidays = holidays[holidays.type != "Event"].drop("type", axis = 1)
regional = holidays[holidays.locale == "Regional"].rename({"locale_name":"state", "description":"holiday_regional"}, axis = 1).drop("locale", axis = 1).drop_duplicates()
national = holidays[holidays.locale == "National"].rename({"description":"holiday_national"}, axis = 1).drop(["locale", "locale_name"], axis = 1).drop_duplicates()
local = holidays[holidays.locale == "Local"].rename({"description":"holiday_local", "locale_name":"city"}, axis = 1).drop("locale", axis = 1).drop_duplicates()



d = pd.concat([train, test]).merge(stores)
d["store_nbr"] = d["store_nbr"].astype("int8")


# National Holidays & Events
#d = pd.merge(d, events, how = "left")
d = pd.merge(d, national, how = "left")
# Regional
d = pd.merge(d, regional, how = "left", on = ["date", "state"])
# Local
d = pd.merge(d, local, how = "left", on = ["date", "city"])

# Work Day: It will be removed when real work day colum created
d = pd.merge(d,  work_day[["date", "type"]].rename({"type":"IsWorkDay"}, axis = 1),how = "left")

# EVENTS
events["events"] =np.where(events.events.str.contains("futbol"), "Futbol", events.events)

def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = df.select_dtypes(["category", "object"]).columns.tolist()
    # categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    df.columns = df.columns.str.replace(" ", "_")
    return df, df.columns.tolist()

events, events_cat = one_hot_encoder(events, nan_as_category=False)
events["events_Dia_de_la_Madre"] = np.where(events.date == "2016-05-08", 1,events["events_Dia_de_la_Madre"])
events = events.drop(239)

d = pd.merge(d, events, how = "left")
d[events_cat] = d[events_cat].fillna(0)

# New features
d["holiday_national_binary"] = np.where(d.holiday_national.notnull(), 1, 0)
d["holiday_local_binary"] = np.where(d.holiday_local.notnull(), 1, 0)
d["holiday_regional_binary"] = np.where(d.holiday_regional.notnull(), 1, 0)

# 
d["national_independence"] = np.where(d.holiday_national.isin(['Batalla de Pichincha',  'Independencia de Cuenca', 'Independencia de Guayaquil', 'Independencia de Guayaquil', 'Primer Grito de Independencia']), 1, 0)
d["local_cantonizacio"] = np.where(d.holiday_local.str.contains("Cantonizacio"), 1, 0)
d["local_fundacion"] = np.where(d.holiday_local.str.contains("Fundacion"), 1, 0)
d["local_independencia"] = np.where(d.holiday_local.str.contains("Independencia"), 1, 0)


holidays, holidays_cat = one_hot_encoder(d[["holiday_national","holiday_regional","holiday_local"]], nan_as_category=False)
d = pd.concat([d.drop(["holiday_national","holiday_regional","holiday_local"], axis = 1),holidays], axis = 1)

he_cols = d.columns[d.columns.str.startswith("events")].tolist() + d.columns[d.columns.str.startswith("holiday")].tolist() + d.columns[d.columns.str.startswith("national")].tolist()+ d.columns[d.columns.str.startswith("local")].tolist()
d[he_cols] = d[he_cols].astype("int8")

d[["family", "city", "state", "type"]] = d[["family", "city", "state", "type"]].astype("category")

del holidays, holidays_cat, work_day, local, regional, national, events, events_cat, tr, tr1, tr2, he_cols
gc.collect()

d.head(10)

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,holiday_local_Fundacion_de_Loja,holiday_local_Fundacion_de_Machala,holiday_local_Fundacion_de_Manta,holiday_local_Fundacion_de_Quito,holiday_local_Fundacion_de_Quito1,holiday_local_Fundacion_de_Riobamba,holiday_local_Fundacion_de_Santo_Domingo,holiday_local_Independencia_de_Ambato,holiday_local_Independencia_de_Guaranda,holiday_local_Independencia_de_Latacunga
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
1,1782,2013-01-02,1,AUTOMOTIVE,2.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
2,3564,2013-01-03,1,AUTOMOTIVE,3.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
3,5346,2013-01-04,1,AUTOMOTIVE,3.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
4,7128,2013-01-05,1,AUTOMOTIVE,5.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
5,8910,2013-01-06,1,AUTOMOTIVE,2.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
6,10692,2013-01-07,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
7,12474,2013-01-08,1,AUTOMOTIVE,2.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
8,14256,2013-01-09,1,AUTOMOTIVE,2.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0
9,16038,2013-01-10,1,AUTOMOTIVE,2.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,0,0,0,0


In [9]:
cities = d.city.unique()
print(cities)

['Quito', 'Santo Domingo', 'Cayambe', 'Latacunga', 'Riobamba', ..., 'Loja', 'Machala', 'Esmeraldas', 'Manta', 'El Carmen']
Length: 22
Categories (22, object): ['Ambato', 'Babahoyo', 'Cayambe', 'Cuenca', ..., 'Quito', 'Riobamba', 'Salinas', 'Santo Domingo']


In [12]:
def read_country(city):
    """
    Convert cities and returns the country
    """
    geolocator = Nominatim(user_agent="google") #user agent can be any user agent 
    location = geolocator.geocode(city, language="en") #specified the language as some countries are in other lanaguages
    country= location.address.split(',')[-1] #split the string based on comma and retruns the last element (country)
    return country


city_country_dict = {}
for city in cities: #comp_location will be your list
    city_country = read_country(city)
    city_country_dict[city] = city_country

print(city_country_dict)

{'Quito': ' Ecuador', 'Santo Domingo': ' Dominican Republic', 'Cayambe': ' Ecuador', 'Latacunga': ' Ecuador', 'Riobamba': ' Ecuador', 'Ibarra': ' Spain', 'Guaranda': ' Colombia', 'Puyo': ' Ecuador', 'Ambato': ' Ecuador', 'Guayaquil': ' Ecuador', 'Salinas': ' United States', 'Daule': ' Ecuador', 'Babahoyo': ' Ecuador', 'Quevedo': ' Spain', 'Playas': ' Ecuador', 'Libertad': ' Argentina', 'Cuenca': ' Spain', 'Loja': ' Ecuador', 'Machala': ' Ecuador', 'Esmeraldas': ' Ecuador', 'Manta': ' Italy', 'El Carmen': ' Colombia'}


In [23]:
city_currency_dict = {}
for city, country in city_country_dict.items():
    country = country[1:]
    country_info = CountryInfo(country)
    city_currency_dict[city] = country_info.currencies()

print(city_currency_dict)

{'Quito': ['USD'], 'Santo Domingo': ['DOP'], 'Cayambe': ['USD'], 'Latacunga': ['USD'], 'Riobamba': ['USD'], 'Ibarra': ['EUR'], 'Guaranda': ['COP'], 'Puyo': ['USD'], 'Ambato': ['USD'], 'Guayaquil': ['USD'], 'Salinas': ['USD', 'USN', 'USS'], 'Daule': ['USD'], 'Babahoyo': ['USD'], 'Quevedo': ['EUR'], 'Playas': ['USD'], 'Libertad': ['ARS'], 'Cuenca': ['EUR'], 'Loja': ['USD'], 'Machala': ['USD'], 'Esmeraldas': ['USD'], 'Manta': ['EUR'], 'El Carmen': ['COP']}


In [25]:
distinct_keys = set()
for value in city_currency_dict.values():
    distinct_keys.update(value)
print(distinct_keys)


{'COP', 'ARS', 'DOP', 'USS', 'USD', 'EUR', 'USN'}


In [29]:
country_info = CountryInfo("Ecuador")
currency = country_info.currencies()
print(currency[0])

USD


In [45]:
currency_parquet = pd.read_parquet('/home/nikl/programming/fhdw/forecasting/ForecastingUniProject/data/forex.parquet')

us_currency = currency_parquet[currency_parquet['currency'] == 'USD']
display(us_currency)

store_dataset = d.copy()
us_currency["date"] = pd.to_datetime(us_currency["date"])
filtered_us_currency = us_currency[(us_currency['date'] >= store_dataset["date"].min()) & (us_currency['date'] <= store_dataset['date'].max())]
filtered_us_currency = filtered_us_currency.reset_index(drop=True)
print(filtered_us_currency.shape)
display(filtered_us_currency)

final_us_currency = filtered_us_currency[filtered_us_currency['slug'] == 'EUR/USD'].drop(['slug', "currency"], axis=1).reset_index(drop=True)
final_us_currency = final_us_currency.rename(columns={"open": "slug_eur_usd_open", "high": "slug_eur_usd_high", "low": "slug_eur_usd_low", "close": "slug_eur_usd_close"})
display(final_us_currency)

merged_df = d.merge(final_us_currency, how="left", on="date")
display(merged_df)

,slug,date,open,high,low,close,currency
190472,JPY/USD,1996-10-30,0.008744,0.008802,0.008735,0.008758,USD
190473,JPY/USD,1996-11-01,0.008811,0.008811,0.008811,0.008811,USD
190474,JPY/USD,1996-11-04,0.008828,0.008853,0.008773,0.008781,USD
190475,JPY/USD,1996-11-05,0.008794,0.008814,0.008747,0.008753,USD
190476,JPY/USD,1996-11-06,0.008754,0.008799,0.008720,0.008776,USD
...,...,...,...,...,...,...,...
1081194,INR/USD,2021-08-24,0.013494,0.013503,0.013474,0.013494,USD
1081195,INR/USD,2021-08-25,0.013487,0.013496,0.013459,0.013487,USD
1081196,INR/USD,2021-08-26,0.013495,0.013501,0.013465,0.013495,USD
1081197,INR/USD,2021-08-27,0.013485,0.013617,0.013454,0.013485,USD


(13379, 7)


/tmp/ipykernel_27876/3474333077.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  us_currency["date"] = pd.to_datetime(us_currency["date"])


,slug,date,open,high,low,close,currency
0,JPY/USD,2013-01-01,0.011542,0.011542,0.011527,0.011531,USD
1,JPY/USD,2013-01-02,0.011546,0.011556,0.011455,0.011547,USD
2,JPY/USD,2013-01-03,0.011461,0.011524,0.011454,0.011458,USD
3,JPY/USD,2013-01-04,0.011402,0.011421,0.011315,0.011397,USD
4,JPY/USD,2013-01-07,0.011337,0.011414,0.011336,0.011338,USD
...,...,...,...,...,...,...,...
13374,INR/USD,2017-08-25,0.015616,0.015657,0.015605,0.015616,USD
13375,INR/USD,2017-08-28,0.015625,0.015681,0.015625,0.015626,USD
13376,INR/USD,2017-08-29,0.015650,0.015676,0.015618,0.015677,USD
13377,INR/USD,2017-08-30,0.015624,0.015656,0.015623,0.015628,USD


,date,slug_eur_usd_open,slug_eur_usd_high,slug_eur_usd_low,slug_eur_usd_close
0,2013-01-01,1.320132,1.322401,1.319592,1.320132
1,2013-01-02,1.319784,1.329805,1.319435,1.319784
2,2013-01-03,1.318392,1.318913,1.308900,1.318218
3,2013-01-04,1.303101,1.306380,1.300052,1.303271
4,2013-01-07,1.306848,1.309929,1.302060,1.306745
...,...,...,...,...,...
1211,2017-08-25,1.179830,1.188425,1.177482,1.180220
1212,2017-08-28,1.195143,1.197605,1.191782,1.195186
1213,2017-08-29,1.196516,1.206695,1.196301,1.196387
1214,2017-08-30,1.197031,1.198610,1.189669,1.197046


,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,holiday_local_Fundacion_de_Quito1,holiday_local_Fundacion_de_Riobamba,holiday_local_Fundacion_de_Santo_Domingo,holiday_local_Independencia_de_Ambato,holiday_local_Independencia_de_Guaranda,holiday_local_Independencia_de_Latacunga,slug_eur_usd_open,slug_eur_usd_high,slug_eur_usd_low,slug_eur_usd_close
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,1.320132,1.322401,1.319592,1.320132
1,1782,2013-01-02,1,AUTOMOTIVE,2.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,1.319784,1.329805,1.319435,1.319784
2,3564,2013-01-03,1,AUTOMOTIVE,3.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,1.318392,1.318913,1.308900,1.318218
3,5346,2013-01-04,1,AUTOMOTIVE,3.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,1.303101,1.306380,1.300052,1.303271
4,7128,2013-01-05,1,AUTOMOTIVE,5.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728947,3029395,2017-08-31,9,POULTRY,NaN,1.0,Quito,Pichincha,B,6,...,0,0,0,0,0,0,1.189004,1.190816,1.182438,1.189343
2728948,3029396,2017-08-31,9,PREPARED FOODS,NaN,0.0,Quito,Pichincha,B,6,...,0,0,0,0,0,0,1.189004,1.190816,1.182438,1.189343
2728949,3029397,2017-08-31,9,PRODUCE,NaN,1.0,Quito,Pichincha,B,6,...,0,0,0,0,0,0,1.189004,1.190816,1.182438,1.189343
2728950,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,Quito,Pichincha,B,6,...,0,0,0,0,0,0,1.189004,1.190816,1.182438,1.189343


In [54]:
inflation_df = pd.read_csv('/home/nikl/programming/fhdw/forecasting/ForecastingUniProject/data/inflation_per_month.csv')
inflation_df["date"] = pd.to_datetime(inflation_df["DATE"])
inflation_df = inflation_df.drop(["DATE"], axis=1)
inflation_df = inflation_df[(inflation_df['date'] >= store_dataset["date"].min()) & (inflation_df['date'] <= store_dataset['date'].max())]
inflation_df = inflation_df.reset_index(drop=True)
inflation_df = inflation_df.rename(columns={"CPIUFDNS_PC1": "inflation_usd"})
display(inflation_df)
new_merged_df = merged_df.merge(inflation_df, how="left", on="date")
display(new_merged_df)

,inflation_usd,date
0,1.57952,2013-01-01
1,1.64096,2013-02-01
2,1.52067,2013-03-01
3,1.54652,2013-04-01
4,1.36582,2013-05-01
5,1.38250,2013-06-01
6,1.44288,2013-07-01
7,1.38796,2013-08-01
8,1.37603,2013-09-01
9,1.27428,2013-10-01


,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,holiday_local_Fundacion_de_Riobamba,holiday_local_Fundacion_de_Santo_Domingo,holiday_local_Independencia_de_Ambato,holiday_local_Independencia_de_Guaranda,holiday_local_Independencia_de_Latacunga,slug_eur_usd_open,slug_eur_usd_high,slug_eur_usd_low,slug_eur_usd_close,inflation_usd
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,1.320132,1.322401,1.319592,1.320132,1.57952
1,1782,2013-01-02,1,AUTOMOTIVE,2.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,1.319784,1.329805,1.319435,1.319784,NaN
2,3564,2013-01-03,1,AUTOMOTIVE,3.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,1.318392,1.318913,1.308900,1.318218,NaN
3,5346,2013-01-04,1,AUTOMOTIVE,3.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,1.303101,1.306380,1.300052,1.303271,NaN
4,7128,2013-01-05,1,AUTOMOTIVE,5.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728947,3029395,2017-08-31,9,POULTRY,NaN,1.0,Quito,Pichincha,B,6,...,0,0,0,0,0,1.189004,1.190816,1.182438,1.189343,NaN
2728948,3029396,2017-08-31,9,PREPARED FOODS,NaN,0.0,Quito,Pichincha,B,6,...,0,0,0,0,0,1.189004,1.190816,1.182438,1.189343,NaN
2728949,3029397,2017-08-31,9,PRODUCE,NaN,1.0,Quito,Pichincha,B,6,...,0,0,0,0,0,1.189004,1.190816,1.182438,1.189343,NaN
2728950,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,Quito,Pichincha,B,6,...,0,0,0,0,0,1.189004,1.190816,1.182438,1.189343,NaN


In [78]:
global_inflation_df = pd.read_csv('/home/nikl/programming/fhdw/forecasting/ForecastingUniProject/data/global_inflation_data.csv')
global_inflation_df = global_inflation_df[global_inflation_df['country_name'] == 'Ecuador']
global_inflation_df = global_inflation_df.transpose()
global_inflation_df = global_inflation_df.iloc[2:]
global_inflation_df = global_inflation_df.reset_index()
global_inflation_df.columns = ['date', 'inflation_ecuador']
global_inflation_df["date"] = pd.to_datetime(global_inflation_df["date"])
global_inflation_df = global_inflation_df[(global_inflation_df['date'] >= store_dataset["date"].min()) & (global_inflation_df['date'] <= store_dataset['date'].max())] 
global_inflation_df = global_inflation_df.reset_index(drop=True)
display(global_inflation_df)
new_new_merged_df = new_merged_df.merge(global_inflation_df, how="left", on="date")
display(new_new_merged_df)

,date,inflation_ecuador
0,2013-01-01,2.7
1,2014-01-01,3.6
2,2015-01-01,4.0
3,2016-01-01,1.7
4,2017-01-01,0.4


,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,holiday_local_Fundacion_de_Santo_Domingo,holiday_local_Independencia_de_Ambato,holiday_local_Independencia_de_Guaranda,holiday_local_Independencia_de_Latacunga,slug_eur_usd_open,slug_eur_usd_high,slug_eur_usd_low,slug_eur_usd_close,inflation_usd,inflation_ecuador
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,1.320132,1.322401,1.319592,1.320132,1.57952,2.7
1,1782,2013-01-02,1,AUTOMOTIVE,2.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,1.319784,1.329805,1.319435,1.319784,NaN,NaN
2,3564,2013-01-03,1,AUTOMOTIVE,3.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,1.318392,1.318913,1.308900,1.318218,NaN,NaN
3,5346,2013-01-04,1,AUTOMOTIVE,3.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,1.303101,1.306380,1.300052,1.303271,NaN,NaN
4,7128,2013-01-05,1,AUTOMOTIVE,5.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728947,3029395,2017-08-31,9,POULTRY,NaN,1.0,Quito,Pichincha,B,6,...,0,0,0,0,1.189004,1.190816,1.182438,1.189343,NaN,NaN
2728948,3029396,2017-08-31,9,PREPARED FOODS,NaN,0.0,Quito,Pichincha,B,6,...,0,0,0,0,1.189004,1.190816,1.182438,1.189343,NaN,NaN
2728949,3029397,2017-08-31,9,PRODUCE,NaN,1.0,Quito,Pichincha,B,6,...,0,0,0,0,1.189004,1.190816,1.182438,1.189343,NaN,NaN
2728950,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,Quito,Pichincha,B,6,...,0,0,0,0,1.189004,1.190816,1.182438,1.189343,NaN,NaN


In [82]:
# Fill missing values using forward fill
new_new_merged_df['slug_eur_usd_open'] = new_new_merged_df['slug_eur_usd_open'].ffill()
new_new_merged_df['slug_eur_usd_high'] = new_new_merged_df['slug_eur_usd_high'].ffill()
new_new_merged_df['slug_eur_usd_low'] = new_new_merged_df['slug_eur_usd_low'].ffill()
new_new_merged_df['slug_eur_usd_close'] = new_new_merged_df['slug_eur_usd_close'].ffill()
new_new_merged_df['inflation_usd'] = new_new_merged_df['inflation_usd'].ffill()
new_new_merged_df['inflation_ecuador'] = new_new_merged_df['inflation_ecuador'].ffill()

print(new_new_merged_df[['slug_eur_usd_open', 'slug_eur_usd_high', 'slug_eur_usd_low', 'slug_eur_usd_close', 'inflation_usd', 'inflation_ecuador']].isnull().sum())
display(new_new_merged_df)


slug_eur_usd_open     0
slug_eur_usd_high     0
slug_eur_usd_low      0
slug_eur_usd_close    0
inflation_usd         0
inflation_ecuador     0
dtype: int64


,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,...,holiday_local_Fundacion_de_Santo_Domingo,holiday_local_Independencia_de_Ambato,holiday_local_Independencia_de_Guaranda,holiday_local_Independencia_de_Latacunga,slug_eur_usd_open,slug_eur_usd_high,slug_eur_usd_low,slug_eur_usd_close,inflation_usd,inflation_ecuador
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,1.320132,1.322401,1.319592,1.320132,1.57952,2.7
1,1782,2013-01-02,1,AUTOMOTIVE,2.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,1.319784,1.329805,1.319435,1.319784,1.57952,2.7
2,3564,2013-01-03,1,AUTOMOTIVE,3.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,1.318392,1.318913,1.308900,1.318218,1.57952,2.7
3,5346,2013-01-04,1,AUTOMOTIVE,3.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,1.303101,1.306380,1.300052,1.303271,1.57952,2.7
4,7128,2013-01-05,1,AUTOMOTIVE,5.0,0.0,Quito,Pichincha,D,13,...,0,0,0,0,1.303101,1.306380,1.300052,1.303271,1.57952,2.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728947,3029395,2017-08-31,9,POULTRY,NaN,1.0,Quito,Pichincha,B,6,...,0,0,0,0,1.189004,1.190816,1.182438,1.189343,1.11982,0.4
2728948,3029396,2017-08-31,9,PREPARED FOODS,NaN,0.0,Quito,Pichincha,B,6,...,0,0,0,0,1.189004,1.190816,1.182438,1.189343,1.11982,0.4
2728949,3029397,2017-08-31,9,PRODUCE,NaN,1.0,Quito,Pichincha,B,6,...,0,0,0,0,1.189004,1.190816,1.182438,1.189343,1.11982,0.4
2728950,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9.0,Quito,Pichincha,B,6,...,0,0,0,0,1.189004,1.190816,1.182438,1.189343,1.11982,0.4
